In [1]:
%pip install pulp

In [2]:
from pulp import *
# Definimos el problema
# Jobshop para 7 máquina y 5 trabajos
J=['J1','J2','J3','J4','J5']
M=['M1','M2','M3','M4','M5','M6','M7']
O=[['M1','M2','M4','M5','M6','M7'],
   ['M6','M3','M5','M7','M2','M1'],
   ['M3','M7','M6','M1','M5','M4'],
   ['M5','M2','M7','M4','M1','M6'],
   ['M7','M1','M5','M3','M4','M2']]
# Tiempos de procesamiento [horas]
p=[[12,8,0,18,7,12,12],
   [16,11,6,0,14,9,6],
   [3,0,12,14,8,14,14],
   [15,13,0,8,9,15,16],
   [10,15,19,6,13,0,11]]
# due dates [horas]
d=[[54],[78],[54],[85],[90]]
# Creación de diccionarios
d=makeDict([J],d) 
p=makeDict([J,M],p)
O=makeDict([J,range(6)],O)

In [3]:
problema1=LpProblem('Jobshop',LpMinimize)
X=LpVariable.dicts('X',((j,i) for j in J for i in M ),lowBound=0,cat='Continuos')
Y=LpVariable.dicts('Y',((v,q,i) for v in J for q in J for i in M),lowBound=0, upBound=1,cat='Binary')
Emax=LpVariable('Emax', lowBound=0, cat='Continuos')

for j in J:
    h=O[j][len(O[j])-1]
    problema1+=d[j]-X[j,h]-p[j][h]<=Emax

for j in J:
  for r in range(len(O[j])-1):
      h=O[j][r]
      h1=O[j][r+1]
      problema1+=X[j,h]+p[j][h]<=X[j,h1]

MM=1000
for i in M:
  for v in J:
    for q in J:
      if(q!=v):
          problema1+=X[v,i]+p[v][i]<=X[q,i]+MM*(1-Y[v,q,i])
          problema1+=X[q,i]+p[q][i]<=X[v,i]+MM*(Y[v,q,i])

problema1+=Emax

problema1.writeLP('e.lp')

problema1.solve()

print("Máxima Anticipación = ", value(problema1.objective))
for j in J:
    for r in O[j]:
        i=O[j][r]
        print(j,".",i,".", value(X[j,i]+p[j][i]-d[j]))

Máxima Anticipación =  0.0
J1 . M1 . -21.0
J1 . M2 . 10.0
J1 . M4 . 28.0
J1 . M5 . 35.0
J1 . M6 . 57.0
J1 . M7 . 69.0
J2 . M6 . -69.0
J2 . M3 . -60.0
J2 . M5 . -21.0
J2 . M7 . -14.0
J2 . M2 . -3.0
J2 . M1 . 18.0
J3 . M3 . -42.0
J3 . M7 . -28.0
J3 . M6 . -14.0
J3 . M1 . -11.0
J3 . M5 . 28.0
J3 . M4 . 56.0
J4 . M5 . -42.0
J4 . M2 . -29.0
J4 . M7 . -5.0
J4 . M4 . 11.0
J4 . M1 . 26.0
J4 . M6 . 41.0
J5 . M7 . -79.0
J5 . M1 . -69.0
J5 . M5 . -56.0
J5 . M3 . -37.0
J5 . M4 . -2.0
J5 . M2 . 13.0
